In [2]:
#import all the libraries
from modsim import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

**Stocks**: Uninfected Lymphocytes (R), Latently Infected Lymphocytes (L), Infected Lymphocytes (E), Virions (V)

**Parameters**: Uninfected Lymphocyte Production (Gamma), Lymphocyte Activation Rate (Tau), Lymphocyte natural death rate (Mu), Infection Rate Modifier (Beta), Proportion of cells latently infected (Rho), Latently Infected Activation Rate (Alpha), Infected Death Rate (Delta), Free Virion Production Rate (Pi), Free Virion Removal Rate (Sigma)

Gamma \* Tau is constant, Infection Rate = Beta * V

**Differential Equations**

dR/dt = Gamma\*Tau - Mu\*R - Beta\*R\*V

dL/dt = Rho\*Beta\*R\*V - Mu\*L - Alpha\*L

dE/dt = (1 - Rho)\*Beta\*R\*V + Alpha\*L - Delta\*E

dV/dt = Pi\*E - Sigma\*V

In [3]:
init = State(R = 0,
             L = 0,
             E = 0,
             V = 0)

,values
R,0
L,0
E,0
V,0


In [4]:
sys = System(gamma = 0,
             tau   = 0,
             mu    = 0,
             beta  = 0,
             rho   = 0,
             alpha = 0,
             delta = 0,
             pi    = 0, 
             sigma = 0)

,values
gamma,0
tau,0
mu,0
beta,0
rho,0
alpha,0
delta,0
pi,0
sigma,0


In [ ]:
def update(system, state):
    R  = state.R
    L  = state.L
    E  = state.E
    V  = state.V
    dR = system.gamma*system.tau - system.mu*R - system.beta*R*V
    dL = system.rho*system.beta*R*V - system.mu*L - system.alpha*L
    dE = (1 - system.rho)*system.beta*R*V + system.alpha*L - system.delta*E
    dV = system.pi*E - system.sigma*V